In [1]:
#Importamos la librería pandas que necesitamos para la lectura, conversión y limpieza de los datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
import re 

#Importamos librerías necesarias para la visualización
# -----------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames
pd.set_option('display.max_rows', None) # para poder visualizar todas las filas de los DataFrames

In [7]:
# Importar librería para la conexión con MySQL
# -----------------------------------------------------------------------
import mysql.connector
from mysql.connector import errorcode



In [2]:
df_HR = pd.read_csv('df_HR_limpio.csv')
df_HR.head()

,Unnamed: 0,Age,Attrition,BusinessTravel,DailyRate,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,DateBirth,RemoteWork
0,0,51,No,Unknown,684.0,6,3,Unknown,"162,0",1,M,51.0,3,5,Research Director,3,Unknown,19537.0,6462,7,No,13,"3,0",3,0,Unknown,5,3.000000,20,15,15,1972,Yes
1,1,52,No,Unknown,699.0,1,4,Life Sciences,"259,0",3,M,65.0,2,5,Manager,3,Unknown,19999.0,5678,0,Unknown,14,"3,0",1,1,"34,0",5,3.000000,33,11,9,1971,1
2,2,42,No,travel_rarely,532.0,4,2,Technical Degree,"319,0",3,M,58.0,3,5,Manager,4,Married,19232.0,4933,1,No,11,"3,0",4,0,"22,0",3,2.766568,22,11,15,1981,1
3,3,47,No,travel_rarely,359.0,2,4,Medical,Unknown,1,F,82.0,3,4,Research Director,3,Married,17169.0,26703,3,Unknown,19,"3,0",2,2,Unknown,2,2.766568,20,5,6,1976,False
4,5,48,No,Unknown,117.0,22,3,Medical,"1900,0",4,F,58.0,3,4,Manager,4,Unknown,17174.0,2437,3,No,11,"3,0",2,1,Unknown,3,3.000000,22,4,7,1975,Yes


In [4]:
df_HR.shape

(1080, 33)

In [8]:
df_Employees = df_HR[["Age", "Gender", "MaritalStatus", "DateBirth", "Education", "EducationField", "Attrition", "BusinessTravel", "DistanceFromHome"]]

df_JobDetails = df_HR[["JobRole", "JobLevel", "NumCompaniesWorked", "TotalWorkingYears", "YearsAtCompany", "YearsSinceLastPromotion", "YearsWithCurrManager"]]

df_Compensation = df_HR[["DailyRate", "HourlyRate", "MonthlyIncome", "MonthlyRate", "StockOptionLevel", "PercentSalaryHike", "OverTime", "TrainingTimesLastYear", "RemoteWork"]]

df_Satisfaction = df_HR[["EnvironmentSatisfaction", "JobInvolvement", "JobSatisfaction", "RelationshipSatisfaction", "WorkLifeBalance", "PerformanceRating"]]

In [9]:
df_empleados = pd.read_csv('Employees.csv')
df_empleados.shape

(1080, 10)

df_employees a lista de tuplas

In [10]:
lista_de_tuplas = [tuple(x) for x in df_Employees.to_records(index=False)] 

In [ ]:
lista_de_tuplas

In [12]:
df_Employees.shape

(1080, 9)

Crear base de datos

In [20]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1')


mycursor = cnx.cursor()
try:
    mycursor.execute("CREATE DATABASE ABC_Corporation")
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

CMySQLCursor: CREATE DATABASE ABC_Corporation


Creacion de tablas

In [22]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='ABC_Corporation')


mycursor = cnx.cursor()
try:
    mycursor.execute("""
        CREATE TABLE Employees (
        EmployeeNumber INT PRIMARY KEY AUTO_INCREMENT,
        Age INT,
        Gender VARCHAR(100),
        MaritalStatus VARCHAR(100),
        DateBirth INT,
        Education INT,
        EducationField VARCHAR(100),
        Attrition VARCHAR(100),
        BusinessTravel VARCHAR(100),
        DistanceFromHome INT
    )
    """)
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

CMySQLCursor: 
        CREATE TABLE Employees (
      ..


Insercion de datos

In [24]:
cnx = mysql.connector.connect(user = 'root',
                              password = 'AlumnaAdalab',
                              host = '127.0.0.1',
                              database = 'ABC_Corporation'
                              )


mycursor = cnx.cursor()
sql = """INSERT INTO Employees (
            Age, Gender, MaritalStatus, DateBirth, Education, EducationField, Attrition, BusinessTravel, DistanceFromHome
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"""
val = [
]

try: 
    mycursor.executemany(sql, val)
    cnx.commit()
    print(mycursor.rowcount, "registro/s insertado/s.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

1000 registro/s insertado/s.


### extraer en lista de tuplas Job Details

In [36]:
df_job_details = pd.read_csv('JobDetails.csv')
df_job_details.shape

(1080, 8)

In [49]:
job_details = [list(x) for x in df_job_details .to_records(index=False)] 

In [50]:
# ver que coincidan los registros de las tuplas con el df
job_details

[[0, ' Research Director ', 5, 7, 'Unknown', 20, 15, 15],
 [1, ' Manager ', 5, 0, '34,0', 33, 11, 9],
 [2, ' Manager ', 5, 1, '22,0', 22, 11, 15],
 [3, ' Research Director ', 4, 3, 'Unknown', 20, 5, 6],
 [5, ' Manager ', 4, 3, 'Unknown', 22, 4, 7],
 [6, ' Sales Executive ', 3, 7, '28,0', 21, 7, 9],
 [7, ' Sales Executive ', 2, 1, '20,0', 20, 11, 6],
 [9, ' Research Director ', 5, 2, '21,0', 18, 0, 11],
 [10, ' Research Director ', 5, 1, 'Unknown', 21, 5, 10],
 [13, ' Research Director ', 5, 1, 'Unknown', 22, 15, 8],
 [15, ' Manager ', 5, 0, 'Unknown', 20, 1, 12],
 [16, ' Research Director ', 5, 7, '40,0', 31, 13, 8],
 [20, ' Healthcare Representative ', 3, 3, '25,0', 23, 14, 4],
 [21, ' Research Director ', 3, 1, '15,0', 15, 8, 12],
 [23, ' Manager ', 4, 1, 'Unknown', 26, 3, 0],
 [27, ' Manufacturing Director ', 2, 9, '18,0', 16, 5, 12],
 [29, ' Sales Executive ', 2, 1, '15,0', 15, 0, 7],
 [30, ' Sales Executive ', 3, 0, 'Unknown', 22, 13, 5],
 [31, ' Healthcare Representative ', 3, 2,

In [47]:
tuplas_job_details

[(0, ' Research Director ', 5, 7, 'Unknown', 20, 15, 15),
 (1, ' Manager ', 5, 0, '34,0', 33, 11, 9),
 (2, ' Manager ', 5, 1, '22,0', 22, 11, 15),
 (3, ' Research Director ', 4, 3, 'Unknown', 20, 5, 6),
 (5, ' Manager ', 4, 3, 'Unknown', 22, 4, 7),
 (6, ' Sales Executive ', 3, 7, '28,0', 21, 7, 9),
 (7, ' Sales Executive ', 2, 1, '20,0', 20, 11, 6),
 (9, ' Research Director ', 5, 2, '21,0', 18, 0, 11),
 (10, ' Research Director ', 5, 1, 'Unknown', 21, 5, 10),
 (13, ' Research Director ', 5, 1, 'Unknown', 22, 15, 8),
 (15, ' Manager ', 5, 0, 'Unknown', 20, 1, 12),
 (16, ' Research Director ', 5, 7, '40,0', 31, 13, 8),
 (20, ' Healthcare Representative ', 3, 3, '25,0', 23, 14, 4),
 (21, ' Research Director ', 3, 1, '15,0', 15, 8, 12),
 (23, ' Manager ', 4, 1, 'Unknown', 26, 3, 0),
 (27, ' Manufacturing Director ', 2, 9, '18,0', 16, 5, 12),
 (29, ' Sales Executive ', 2, 1, '15,0', 15, 0, 7),
 (30, ' Sales Executive ', 3, 0, 'Unknown', 22, 13, 5),
 (31, ' Healthcare Representative ', 3, 2,

### Crear tabla JobDetails

In [45]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='ABC_Corporation')


mycursor = cnx.cursor()
try:
    mycursor.execute("""
        CREATE TABLE JobDetails (
        JobDetailID INT PRIMARY KEY AUTO_INCREMENT,
        JobRole VARCHAR(100),
        JobLevel INT,
        NumCompaniesWorked INT,
        TotalWorkingYears VARCHAR(100),
        YearsAtCompany INT,
        YearsSinceLastPromotion INT,
        YearsWithCurrManager INT,
        EmployeeNumber INT,
        FOREIGN KEY (EmployeeNumber) REFERENCES Employees(EmployeeNumber)
    )
    """)
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

1050 (42S01): Table 'jobdetails' already exists
Error Code: 1050
SQLSTATE 42S01
Message Table 'jobdetails' already exists


### Crear tabla Compensation

In [60]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='ABC_Corporation')


mycursor = cnx.cursor()
try:
    mycursor.execute("""
        CREATE TABLE Compensation (
        CompensationID INT PRIMARY KEY AUTO_INCREMENT,
        EmployeeNumber INT,
        DailyRate INT,
        HourlyRate INT,
        MonthlyIncome INT,
        MonthlyRate INT,
        StockOptionLevel INT,
        PercentSalaryHike INT,
        OverTime VARCHAR(100),
        TrainingTimesLastYear INT,
        RemoteWork VARCHAR(100),
        FOREIGN KEY (EmployeeNumber) REFERENCES Employees(EmployeeNumber)
    )
    """)
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

CMySQLCursor: 
        CREATE TABLE Compensation (
   ..


### Creacion tabla Satisfaction

In [63]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='ABC_Corporation')


mycursor = cnx.cursor()
try:
    mycursor.execute("""
        CREATE TABLE Satisfaction (
        SatisfactionID INT PRIMARY KEY AUTO_INCREMENT,
        EmployeeNumber INT,
        EnvironmentSatisfaction INT,
        JobInvolvement INT,
        JobSatisfaction INT,
        RelationshipSatisfaction INT,
        WorkLifeBalance INT,
        PerformanceRating VARCHAR(100),
        FOREIGN KEY (EmployeeNumber) REFERENCES Employees(EmployeeNumber)
    )
    """)
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

1050 (42S01): Table 'satisfaction' already exists
Error Code: 1050
SQLSTATE 42S01
Message Table 'satisfaction' already exists
